In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [19]:
data = pd.read_csv('транзакции_test.csv', names = ['email', 'number'])
data

,email,number
0,user_11799@post.ru,8.801111e+12
1,user_5682@post.ru,8.801111e+12
2,user_14735@post.ru,8.801111e+11
3,user_11725@post.ru,8.801111e+11
4,user_7948@post.ru,8.801111e+11
...,...,...
12081,user_9750@post.ru,8.801111e+11
12082,user_7735@post.ru,8.801111e+11
12083,user_520@post.ru,8.801111e+11
12084,user_5325@post.ru,8.801111e+12


Уберем все NaN, заменив их на -1

In [20]:
data = data.fillna(-1)
data

,email,number
0,user_11799@post.ru,8.801111e+12
1,user_5682@post.ru,8.801111e+12
2,user_14735@post.ru,8.801111e+11
3,user_11725@post.ru,8.801111e+11
4,user_7948@post.ru,8.801111e+11
...,...,...
12081,user_9750@post.ru,8.801111e+11
12082,user_7735@post.ru,8.801111e+11
12083,user_520@post.ru,8.801111e+11
12084,user_5325@post.ru,8.801111e+12


Теперь нужно убрать повторяющиеся строки и посчитать количество повторений для каждой строки

In [21]:
data = data.groupby(data.columns.tolist(), as_index= False).size()
data

,email,number,size
0,-1,8.801111e+11,1
1,-1,8.801111e+11,1
2,user_0@post.ru,8.801111e+11,1
3,user_0@post.ru,8.801111e+11,1
4,user_0@post.ru,8.801111e+12,1
...,...,...,...
12081,user_9990@post.ru,8.801111e+11,1
12082,user_9990@post.ru,8.801111e+12,1
12083,user_9990@post.ru,8.801111e+12,1
12084,user_9993@post.ru,8.801111e+11,1


Данную структуру можно рассматривать как список ребер графа, веса ребер -- значения повторяющихся строк. Тогда необходимо найти все компоненты связности этого графа. Проблема возникает с -1, так как она может относится к нескольким компонентами связности. Сделаем обход графа в глубину, игнорируя попадание в -1. Количество транзакций каждого пользователя -- сумма значений ребер каждой компоненты. Можно использовать networkx для простоты

In [22]:
g = nx.from_pandas_edgelist(data,source='email', target='number',edge_attr='size',)
print(g)

Graph with 13929 nodes and 12086 edges


In [23]:
for i in g.nodes:
    g.nodes[i]['visited'] = False

In [25]:
def dfs1(node): #адаптированный dfs, игнорирующий попадание в -1 и считающий сумму ребер
    global visited
    global transactions
    visited.append(node)
    g.nodes[node]['visited'] = True

    for neighbor, edge_weight in g[node].items():
        if g.nodes[neighbor]['visited'] == False:
            transactions += edge_weight['size']

    for neighbor, edge_weight in g[node].items():
        if neighbor == -1:
            continue
        if g.nodes[neighbor]['visited'] == False:
            dfs1(neighbor)

In [26]:
tr = []
for i in g.nodes:
    visited = []
    if i != -1:
        if g.nodes[i]['visited'] == False:
            transactions = 0
            dfs1(i)
            tr.append(transactions)
tr #ответ

[1,
 1,
 5191,
 2,
 2,
 8,
 10,
 2,
 2,
 2,
 3,
 5,
 30,
 23,
 7,
 16,
 8,
 1,
 2,
 3,
 3,
 261,
 5,
 15,
 2,
 1,
 1,
 13,
 4,
 3,
 67,
 20,
 1,
 23,
 46,
 4,
 1,
 6,
 4,
 2,
 8,
 5,
 2,
 3,
 3,
 13,
 8,
 30,
 1,
 7,
 6,
 4,
 7,
 8,
 3,
 5,
 11,
 3,
 2,
 9,
 5,
 35,
 28,
 8,
 33,
 2,
 16,
 1,
 3,
 3,
 7,
 1,
 7,
 3,
 3,
 3,
 8,
 8,
 3,
 8,
 68,
 5,
 2,
 7,
 4,
 7,
 15,
 3,
 4,
 1,
 2,
 1,
 6,
 1,
 3,
 4,
 23,
 19,
 2,
 2,
 2,
 1,
 2,
 36,
 31,
 4,
 1,
 4,
 13,
 4,
 2,
 2,
 1,
 7,
 3,
 7,
 1,
 8,
 11,
 8,
 1,
 1,
 4,
 8,
 6,
 3,
 5,
 3,
 10,
 18,
 5,
 2,
 1,
 3,
 5,
 48,
 3,
 5,
 8,
 30,
 2,
 17,
 7,
 4,
 1,
 2,
 7,
 3,
 3,
 4,
 1,
 1,
 3,
 2,
 1,
 7,
 1,
 7,
 27,
 1,
 12,
 40,
 9,
 6,
 2,
 5,
 2,
 5,
 1,
 7,
 1,
 4,
 8,
 27,
 3,
 8,
 3,
 2,
 7,
 5,
 8,
 8,
 3,
 9,
 20,
 5,
 2,
 4,
 5,
 3,
 15,
 9,
 2,
 1,
 2,
 4,
 2,
 1,
 3,
 3,
 1,
 4,
 3,
 3,
 1,
 3,
 1,
 2,
 2,
 6,
 3,
 3,
 8,
 4,
 1,
 15,
 3,
 2,
 1,
 3,
 5,
 3,
 3,
 1,
 20,
 1,
 1,
 5,
 3,
 2,
 6,
 17,
 1,
 4,
 2,
 1,
 21,
 6,
 3,

In [27]:
for i in g.nodes(): #проверка, что все вершины, кроме -1 посещены
    if g.nodes[i]['visited'] == False:
        print(i)

-1


In [28]:
s = 0 #Проверка того, что количество транзакций каждого пользователя в сумме дает общее количество транзакций
for i in range(len(tr)):
    s += tr[i]
s

12086